In [1]:
import tensorflow as tf
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten, MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, ReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageOps
import os
import time
import matplotlib.pyplot as plt
import cv2

Using TensorFlow backend.


In [2]:
DATA_PATH = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060'
image_size = 256

# insert images as np.array
training_images_path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060/images_256_256_MD.npy'

# insert classes as np.array Users/10539060/images_256_256_MD.npy
training_labels_path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpacganfirstattempt/code/Users/10539060/labels_256_256_MD.npy'

In [11]:
# define the standalone discriminator model
def define_discriminator(in_shape=(256,256,3), n_classes=4):
        # label input
        in_label = Input(shape=(1,))
        # embedding for categorical input
        li = Embedding(n_classes, 35)(in_label)
        # scale up to image dimensions with linear activation
        n_nodes = in_shape[0] * in_shape[1]
        li = Dense(n_nodes)(li)
        # reshape to additional channel
        li = Reshape((in_shape[0], in_shape[1], 1))(li)
        # image input
        in_image = Input(shape=in_shape)
        # concat label as a channel
        merge = Concatenate()([in_image, li])
        fe = Conv2D(256, (3,3), strides=(2,2), padding='same')(merge)
        fe = LeakyReLU(alpha=0.2)(fe) 
        # downsample
        fe = Conv2D(512, (3,3), strides=(2,2), padding='same')(fe)
        fe = LeakyReLU(alpha=0.2)(fe)
        # flatten feature maps
        fe = Flatten()(fe)
        # dropout
        fe = Dropout(0.4)(fe)
        # output
        out_layer = Dense(1, activation='sigmoid')(fe)
        # define model
        model = Model([in_image, in_label], out_layer)
        # compile model
        opt = Adam(lr=0.0002, beta_1=0.5) 
        model.summary()
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model

# define the standalone generator model
def define_generator(latent_dim, size, n_classes=4):
    # label input
        in_label = Input(shape=(1,))
        # embedding for categorical input
        li = Embedding(n_classes, 35)(in_label)
        # linear multiplication
        n_nodes = int(size/4) * int(size/4)
        li = Dense(n_nodes)(li)
        # reshape to additional channel
        li = Reshape((int(size/4), int(size/4), 1))(li)
        # image generator input
        in_lat = Input(shape=(latent_dim,))
        # foundation
        n_nodes = 128 * int(size/4) * int(size/4)
        gen = Dense(n_nodes)(in_lat)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Reshape((int(size/4), int(size/4), 128))(gen)
        # merge image gen and label input
        merge = Concatenate()([gen, li])
        # first upsample
        gen = Conv2DTranspose(512, (3,3), strides=(2, 2), padding='same')(merge)  
        gen = LeakyReLU(alpha=0.3)(gen)
        #gen = UpSampling2D()(gen)
        # second upsample
        gen = Conv2DTranspose(256, (3,3), strides=(2, 2), padding='same')(gen) 
        gen = LeakyReLU(alpha=0.3)(gen)
        #gen = UpSampling2D()(gen)
        # output
        out_layer = Conv2D(3, (7,7), activation='tanh', padding='same')(gen)
        # define model
        model = Model([in_lat, in_label], out_layer)
        model.summary()
        return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label= g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [12]:
def load_real_samples():
  trainX = np.load(training_images_path, allow_pickle = True)
  trainy = np.load(training_labels_path, allow_pickle = True)
  #Adding noise to input data
  X = trainX.astype('float32')
  X = (X - 127.5) / 127.5 + np.random.normal(0, 0.01, (trainX.shape[0],trainX.shape[1],trainX.shape[2],trainX.shape[3]))
  #From str labels to integers
  for i in range(len(trainy)):
    if trainy[i] == 'Happy':
      trainy[i] = np.int(0)
    if trainy[i] == 'Angry':
      trainy[i] = np.int(1)
    if trainy[i] == 'Sad':
      trainy[i] = np.int(2)
    if trainy[i] == 'Relaxed':
      trainy[i] = np.int(3)
  trainy = trainy.astype('float32')
  return[X, trainy]

In [13]:
# # select real samples
def generate_real_samples(dataset, n_samples):
  # split into images and labels
  images, labels = dataset
  # choose random instances
  ix = randint(0, images.shape[0], n_samples)
  # select images and labels
  X, labels = images[ix], labels[ix]
  # generate class labels
  y = ones((n_samples, 1))
  # labels switching
  for i in range(int(n_samples/3)):
    y[3*i] = 0 
  return [X, labels], y

In [14]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=4):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

In [15]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
  # generate points in latent space
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  # predict outputs
  images = generator.predict([z_input, labels_input])
  # create class labels
  y = zeros((n_samples, 1))
  #labels switching
  for i in range(int(n_samples/3)):
    y[3*i] = 1
  return [images, labels_input], y

In [16]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=25, n_batch=16): #batch era 128
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      # get randomly selected 'real' samples
      [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
      # update discriminator model weights
      d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
      # generate 'fake' examples
      [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator model weights
      d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
      # prepare points in latent space as input for the generator
      [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
      # summarize loss on this batch
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
        (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))

In [17]:
# size of the latent space
latent_dim = 16
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim, image_size)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 35)        140         input_5[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 1, 65536)     2359296     embedding_3[0][0]                
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
____________________________________________________________________________________________

In [18]:
# train and save model
train(g_model, d_model, gan_model, dataset, latent_dim)
g_model.save(os.path.join(DATA_PATH, f'generator_{image_size}_100_new.h5'))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/

>1, 1/465, d1=0.701, d2=0.697 g=0.707
>1, 2/465, d1=1.788, d2=0.683 g=0.728
>1, 3/465, d1=0.388, d2=0.684 g=0.724
>1, 4/465, d1=1.006, d2=0.684 g=0.724
>1, 5/465, d1=1.116, d2=0.681 g=0.726
>1, 6/465, d1=1.119, d2=0.688 g=0.716
>1, 7/465, d1=0.722, d2=0.682 g=0.761
>1, 8/465, d1=1.064, d2=0.668 g=0.780
>1, 9/465, d1=0.650, d2=0.658 g=0.811
>1, 10/465, d1=0.665, d2=0.651 g=0.845
>1, 11/465, d1=0.675, d2=0.640 g=0.915
>1, 12/465, d1=1.366, d2=0.662 g=0.785
>1, 13/465, d1=0.500, d2=0.677 g=0.865
>1, 14/465, d1=0.875, d2=0.589 g=1.287
>1, 15/465, d1=1.316, d2=0.689 g=0.659
>1, 16/465, d1=0.732, d2=0.776 g=0.705
>1, 17/465, d1=0.683, d2=0.755 g=0.826
>1, 18/465, d1=0.664, d2=0.724 g=0.813
>1, 19/465, d1=0.792, d2=0.670 g=1.075
>1, 20/465, d1=0.656, d2=0.676 g=0.961
>1, 21/465, d1=1.071, d2=0.616 g=1.151
>1, 22/465, d1=0.645, d2=0.644 g=0.990
>1, 23/465, d1=1.655, d2=0.648 g=0.982
>1, 24/465, d1=1.112, d2=0.680 g=0.824
>1, 25/465, d1=0.785, d2=0.695 g=0.805
>1, 26/465, d1=0.551, d2=0.627 g=1

In [20]:
g_model.save(os.path.join(DATA_PATH, f'generator_{image_size}_100_new.h5'))